In [1]:
import pandas as pd

# Load datasets
train_df = pd.read_csv("/content/usjobs_train.csv")
test_df = pd.read_csv("/content/usjobs_test.csv")

# Preview the data
print("Train Shape:", train_df.shape)
print("Test Shape:", test_df.shape)

# Show missing value percentages
missing_train = train_df.isnull().mean().sort_values(ascending=False)
missing_test = test_df.isnull().mean().sort_values(ascending=False)

print("\nMissing values in Train:\n", missing_train)
print("\nMissing values in Test:\n", missing_test)

# Show sample data
print("\nTrain Sample:\n", train_df.head(3))
print("\nTest Sample:\n", test_df.head(3))

Train Shape: (33248, 21)
Test Shape: (22166, 20)

Missing values in Train:
 Director_Score      0.659408
Profile             0.634835
Director            0.625150
Remote              0.581058
Revenue             0.550950
URL                 0.482224
Employee            0.384955
Reviews             0.263535
Company_Score       0.263535
Sector_Group        0.216975
Sector              0.216975
City                0.115014
State               0.093600
Location            0.000391
Company             0.000271
ID                  0.000000
Jobs_Group          0.000000
Job                 0.000000
Skills              0.000000
Frecuency_Salary    0.000000
Mean_Salary         0.000000
dtype: float64

Missing values in Test:
 Director_Score      0.652531
Profile             0.634350
Director            0.616304
Remote              0.578453
Revenue             0.542362
URL                 0.474556
Employee            0.379771
Company_Score       0.262925
Reviews             0.262925
Sector       

In [2]:
cols_to_drop = ['Director', 'Director_Score', 'URL', 'Profile']

train_df = train_df.drop(columns=cols_to_drop)
test_df = test_df.drop(columns=cols_to_drop)

In [3]:
# Filter train data for yearly salaries only
train_df = train_df[train_df['Frecuency_Salary'] == 'year']
test_df = test_df[test_df['Frecuency_Salary'] == 'year']

In [4]:
# Fill missing categorical values
categorical_cols = ['Remote', 'Sector', 'Sector_Group', 'Revenue', 'Employee', 'City', 'State', 'Company']
for col in categorical_cols:
    train_df[col] = train_df[col].fillna('Unknown')
    test_df[col] = test_df[col].fillna('Unknown')

# Fill missing numeric values
numeric_cols = ['Company_Score', 'Reviews']
for col in numeric_cols:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(test_df[col].median())

In [5]:
# Drop only columns that exist
cols_to_drop = ['Director', 'Director_Score', 'URL', 'Profile']
existing_cols_to_drop = [col for col in cols_to_drop if col in train_df.columns]

train_df = train_df.drop(columns=existing_cols_to_drop)
test_df = test_df.drop(columns=existing_cols_to_drop)

In [6]:
from sklearn.preprocessing import OneHotEncoder

# Step 1: One-hot encode categorical features
categorical_cols = ['Remote', 'Revenue', 'Employee', 'Sector', 'Sector_Group', 'State', 'Jobs_Group']
all_df = pd.concat([train_df, test_df], axis=0)
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_cat = pd.DataFrame(ohe.fit_transform(all_df[categorical_cols]), columns=ohe.get_feature_names_out(categorical_cols))
encoded_cat.index = all_df.index
all_df = pd.concat([all_df.drop(columns=categorical_cols), encoded_cat], axis=1)

In [7]:
import ast
from collections import Counter

# Convert stringified lists to real lists
all_df['Skills'] = all_df['Skills'].apply(ast.literal_eval)

# Count top 20 most common skills
skill_counts = Counter(skill for skills in all_df['Skills'] for skill in skills)
top_skills = [skill for skill, count in skill_counts.most_common(20)]

# Create binary features for top skills
for skill in top_skills:
    all_df[f'skill_{skill}'] = all_df['Skills'].apply(lambda x: int(skill in x))

# Drop original Skills column
all_df = all_df.drop(columns=['Skills'])

In [8]:
# Re-separate train and test
train_final = all_df[all_df['Mean_Salary'].notnull()]
test_final = all_df[all_df['Mean_Salary'].isnull()]

# Define features and target
X = train_final.drop(columns=['ID', 'Job', 'Company', 'Location', 'City', 'Mean_Salary', 'Frecuency_Salary'])
y = train_final['Mean_Salary']
X_test = test_final.drop(columns=['ID', 'Job', 'Company', 'Location', 'City', 'Mean_Salary', 'Frecuency_Salary'])

In [9]:
from sklearn.preprocessing import OneHotEncoder
import ast
from collections import Counter
import sklearn

# Step 1: One-hot encode categorical features (version-safe)
categorical_cols = ['Remote', 'Revenue', 'Employee', 'Sector', 'Sector_Group', 'State', 'Jobs_Group']

# Combine train and test
all_df = pd.concat([train_df, test_df], axis=0)

# Create encoder with correct parameters based on sklearn version
if sklearn.__version__ >= '1.2':
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
else:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit and transform
encoded_cat = pd.DataFrame(ohe.fit_transform(all_df[categorical_cols]),
                           columns=ohe.get_feature_names_out(categorical_cols))
encoded_cat.index = all_df.index

# Add back to dataframe
all_df = pd.concat([all_df.drop(columns=categorical_cols), encoded_cat], axis=1)

# Step 2: Process Skills column
# Convert string lists to real Python lists
all_df['Skills'] = all_df['Skills'].apply(ast.literal_eval)

# Count top 20 skills
skill_counts = Counter(skill for skills in all_df['Skills'] for skill in skills)
top_skills = [skill for skill, count in skill_counts.most_common(20)]

# Create binary feature for each top skill
for skill in top_skills:
    all_df[f'skill_{skill}'] = all_df['Skills'].apply(lambda x: int(skill in x))

# Drop original skills column
all_df = all_df.drop(columns=['Skills'])

# Step 3: Re-split datasets
train_final = all_df[all_df['Mean_Salary'].notnull()]
test_final = all_df[all_df['Mean_Salary'].isnull()]

# Features and target
X = train_final.drop(columns=['ID', 'Job', 'Company', 'Location', 'City', 'Mean_Salary', 'Frecuency_Salary'])
y = train_final['Mean_Salary']
X_test = test_final.drop(columns=['ID', 'Job', 'Company', 'Location', 'City', 'Mean_Salary', 'Frecuency_Salary'])

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Initialize model
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Evaluate model using cross-validation (R² Score)
scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f'Cross-validated R² scores: {scores}')
print(f'Mean R² score: {scores.mean():.4f}')

# Fit the model on the full training data
model.fit(X, y)

KeyboardInterrupt: 

In [ ]:
# Predict salary for test data
test_preds = model.predict(X_test)

In [ ]:
print("Original test_df shape:", test_df.shape)
print("Processed test_final shape:", test_final.shape)

In [ ]:
print("Skills in train_df:", 'Skills' in train_df.columns)
print("Skills in test_df:", 'Skills' in test_df.columns)
print("Skills in all_df:", 'Skills' in all_df.columns)

In [ ]:
import ast

# Safe parsing of 'Skills'
def safe_parse(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

# Apply parsing to both dataframes
train_df['Skills'] = train_df['Skills'].fillna("[]").apply(safe_parse)
test_df['Skills'] = test_df['Skills'].fillna("[]").apply(safe_parse)

In [ ]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [ ]:
from collections import Counter

# Count all skills
skill_counter = Counter()
for skills in all_df['Skills']:
    skill_counter.update(skills)

# Use top 100 frequent skills (you can change this number)
top_skills = [skill for skill, _ in skill_counter.most_common(100)]

# Create binary features for top skills
for skill in top_skills:
    all_df[f'skill_{skill}'] = all_df['Skills'].apply(lambda x: int(skill in x))

In [ ]:
all_df.drop(columns=['Skills'], inplace=True)

In [ ]:
train_final = all_df.iloc[:len(train_df)].copy()
test_final = all_df.iloc[len(train_df):].copy()

X = train_final.drop(columns=['Mean_Salary', 'ID'])
y = train_final['Mean_Salary']
X_test = test_final.drop(columns=['ID'])

In [ ]:
# Show non-numeric columns in X
non_numeric_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Non-numeric columns in X:", non_numeric_cols)

In [ ]:
X = X.drop(columns=['Job', 'Company', 'Location', 'City'])
X_test = X_test.drop(columns=['Job', 'Company', 'Location', 'City'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Concatenate train and test again to ensure consistent encoding
combined = pd.concat([X, X_test], axis=0)

# One-hot encode remaining categorical columns
categorical_cols = combined.select_dtypes(include=['object']).columns.tolist()
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # for scikit-learn >=1.2 use `sparse_output`

encoded = pd.DataFrame(ohe.fit_transform(combined[categorical_cols]), columns=ohe.get_feature_names_out(categorical_cols))

# Merge back with numeric features
combined = combined.drop(columns=categorical_cols).reset_index(drop=True)
encoded = encoded.reset_index(drop=True)
final_data = pd.concat([combined, encoded], axis=1)

# Re-split
X = final_data.iloc[:len(X)]
X_test = final_data.iloc[len(X):]

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)
test_preds = model.predict(X_test)

In [ ]:
print("Original test_df shape:", test_df.shape)
print("Processed X_test shape:", X_test.shape)
print("Length of predictions:", len(test_preds))
print("Submission shape:", submission.shape)

In [ ]:
# Before any cleaning
test_ids = test_df['ID'].copy()

In [ ]:
test_ids = test_df['ID'].copy()

In [ ]:
train_df.fillna(-1, inplace=True)
test_df.fillna(-1, inplace=True)

In [ ]:
test_preds = model.predict(X_test)

In [ ]:
# Load sample submission
submission = pd.read_csv('/content/usjobs_sample_submission.csv')

# Reset test_ids to align with X_test
test_ids = test_df['ID'].reset_index(drop=True)

# Reset predictions to align with test_ids
preds_df = pd.DataFrame({
    'ID': test_ids,
    'Mean_Salary': test_preds
})

# Merge with submission on 'ID'
submission = submission.drop(columns=['Mean_Salary'], errors='ignore')
submission = submission.merge(preds_df, on='ID', how='left')

# Fill any missing predictions with median
submission['Mean_Salary'] = submission['Mean_Salary'].fillna(submission['Mean_Salary'].median())

# Save final submission
submission.to_csv('usjobs_final_submission.csv', index=False)

print("✅ Submission file saved: usjobs_final_submission.csv")

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from math import sqrt

# 📂 Load the data
train_df = pd.read_csv('/content/usjobs_train.csv')
test_df = pd.read_csv('/content/usjobs_test.csv')
submission = pd.read_csv('/content/usjobs_sample_submission.csv')

# 🧹 Drop sparse or unhelpful columns
cols_to_drop = ['Director', 'Director_Score', 'URL', 'Profile']
train_df = train_df.drop(columns=[col for col in cols_to_drop if col in train_df.columns])
test_df = test_df.drop(columns=[col for col in cols_to_drop if col in test_df.columns])

# 🔍 Combine for consistent preprocessing
all_df = pd.concat([train_df.drop(columns=['Mean_Salary']), test_df], axis=0).reset_index(drop=True)

# 🔄 Fill missing categorical values
categorical_cols = ['Remote', 'Revenue', 'Employee', 'Sector', 'Sector_Group', 'State', 'Jobs_Group']
for col in categorical_cols:
    all_df[col] = all_df[col].fillna('Unknown')

# 🧠 One-Hot Encoding
ohe = OneHotEncoder(handle_unknown='ignore')
encoded_cat = ohe.fit_transform(all_df[categorical_cols])

# 🧾 TF-IDF on Skills
all_df['Skills'] = all_df['Skills'].fillna('')
tfidf = TfidfVectorizer(max_features=100)
skills_tfidf = tfidf.fit_transform(all_df['Skills'])

# 🧠 Feature Matrix
X_all = hstack([encoded_cat, skills_tfidf])

# 🔀 Split back
X_train = X_all[:len(train_df)]
X_test = X_all[len(train_df):]
y = train_df['Mean_Salary']

# 🧪 Validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

# 🌲 Model Training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 📈 Validation Performance
val_preds = model.predict(X_val)
rmse = sqrt(mean_squared_error(y_val, val_preds))
print(f"Validation RMSE: {rmse:.2f}")

# 🧪 Predict on Test
test_preds = model.predict(X_test)

# 📤 Prepare Submission
if len(test_preds) != len(submission):
    print("Mismatch! Adjusting prediction length...")
    test_preds = test_preds[:len(submission)]  # Safe trim

submission['Mean_Salary'] = test_preds
submission.to_csv('usjobs_final_submission.csv', index=False)
print("✅ Submission file 'usjobs_final_submission.csv' is ready.")

Validation RMSE: 32050.41
✅ Submission file 'usjobs_final_submission.csv' is ready.


In [12]:
pip install streamlit scikit-learn pandas scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [15]:
import pickle

# After training...
pickle.dump(model, open('rf_model.pkl', 'wb'))
pickle.dump(ohe, open('ohe_encoder.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf_vectorizer.pkl', 'wb'))

In [16]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# --- Load Pretrained Objects ---
@st.cache_resource
def load_model_and_encoders():
    # Load model
    model = pickle.load(open('rf_model.pkl', 'rb'))
    ohe = pickle.load(open('ohe_encoder.pkl', 'rb'))
    tfidf = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))
    return model, ohe, tfidf

model, ohe, tfidf = load_model_and_encoders()

# --- Streamlit UI ---
st.title("💼 US Job Salary Predictor")
st.markdown("Enter job details to estimate the Mean Salary (USD).")

# --- Input Fields ---
remote = st.selectbox("Remote", ['Yes', 'No', 'Unknown'])
revenue = st.selectbox("Company Revenue", ['Unknown', '<$1M', '$1M-$10M', '$10M-$100M', '$100M-$1B', '>$1B'])
employee = st.selectbox("Employee Size", ['Unknown', '1-10', '11-50', '51-200', '201-500', '501-1000', '1001-5000', '5001-10000', '10000+'])
sector = st.selectbox("Sector", ['Information Technology', 'Finance', 'Healthcare', 'Retail', 'Manufacturing', 'Unknown'])
sector_group = st.selectbox("Sector Group", ['Tech', 'Business', 'Healthcare', 'Unknown'])
state = st.selectbox("Job State", ['CA', 'NY', 'TX', 'FL', 'IL', 'Remote', 'Unknown'])
jobs_group = st.selectbox("Job Group", ['Engineering', 'Marketing', 'Sales', 'Operations', 'Unknown'])
skills = st.text_area("Skills / Keywords", "Python, SQL, Excel")

# --- Predict Button ---
if st.button("Predict Salary"):
    # Format input
    input_df = pd.DataFrame([{
        'Remote': remote,
        'Revenue': revenue,
        'Employee': employee,
        'Sector': sector,
        'Sector_Group': sector_group,
        'State': state,
        'Jobs_Group': jobs_group,
        'Skills': skills
    }])

    # One-hot encode categoricals
    cat_features = ohe.transform(input_df[['Remote', 'Revenue', 'Employee', 'Sector', 'Sector_Group', 'State', 'Jobs_Group']])

    # TF-IDF for Skills
    skill_features = tfidf.transform(input_df['Skills'])

    # Combine features
    final_input = hstack([cat_features, skill_features])

    # Predict
    prediction = model.predict(final_input)[0]
    st.success(f"💰 Estimated Mean Salary: **${prediction:,.2f} USD**")

2025-07-23 13:32:31.193 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:32:31.269 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-23 13:32:31.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:32:31.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:32:31.271 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:32:31.499 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:32:31.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:32:31.501 Thread 'MainThread': mi

In [17]:
%%writefile my_app.py

import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# --- Load Pretrained Objects ---
@st.cache_resource
def load_model_and_encoders():
    # Load model
    model = pickle.load(open('rf_model.pkl', 'rb'))
    ohe = pickle.load(open('ohe_encoder.pkl', 'rb'))
    tfidf = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))
    return model, ohe, tfidf

model, ohe, tfidf = load_model_and_encoders()

# --- Streamlit UI ---
st.title("💼 US Job Salary Predictor")
st.markdown("Enter job details to estimate the Mean Salary (USD).")

# --- Input Fields ---
remote = st.selectbox("Remote", ['Yes', 'No', 'Unknown'])
revenue = st.selectbox("Company Revenue", ['Unknown', '<$1M', '$1M-$10M', '$10M-$100M', '$100M-$1B', '>$1B'])
employee = st.selectbox("Employee Size", ['Unknown', '1-10', '11-50', '51-200', '201-500', '501-1000', '1001-5000', '5001-10000', '10000+'])
sector = st.selectbox("Sector", ['Information Technology', 'Finance', 'Healthcare', 'Retail', 'Manufacturing', 'Unknown'])
sector_group = st.selectbox("Sector Group", ['Tech', 'Business', 'Healthcare', 'Unknown'])
state = st.selectbox("Job State", ['CA', 'NY', 'TX', 'FL', 'IL', 'Remote', 'Unknown'])
jobs_group = st.selectbox("Job Group", ['Engineering', 'Marketing', 'Sales', 'Operations', 'Unknown'])
skills = st.text_area("Skills / Keywords", "Python, SQL, Excel")

# --- Predict Button ---
if st.button("Predict Salary"):
    # Format input
    input_df = pd.DataFrame([{
        'Remote': remote,
        'Revenue': revenue,
        'Employee': employee,
        'Sector': sector,
        'Sector_Group': sector_group,
        'State': state,
        'Jobs_Group': jobs_group,
        'Skills': skills
    }])

    # One-hot encode categoricals
    cat_features = ohe.transform(input_df[['Remote', 'Revenue', 'Employee', 'Sector', 'Sector_Group', 'State', 'Jobs_Group']])

    # TF-IDF for Skills
    skill_features = tfidf.transform(input_df['Skills'])

    # Combine features
    final_input = hstack([cat_features, skill_features])

    # Predict
    prediction = model.predict(final_input)[0]
    st.success(f"💰 Estimated Mean Salary: **${prediction:,.2f} USD**")

Writing my_app.py


In [18]:
!wget -q -O - ipv4.icanhazip.com

35.230.121.69


In [19]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [20]:
!npm audit fix --force

npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼⠴npm warn audit Updating localtunnel to 1.8.3, which is a SemVer major change.
⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇npm warn deprecated cryptiles@2.0.5: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
⠇npm warn deprecated sntp@1.0.9: This module moved to @hapi/sntp. Please make sure to switch over as this distribution is no longer supported and may contain bugs and critical security issues.
⠏⠋npm warn deprecated har-validator@4.2.1: this library is no longer supported
⠋npm warn deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/

In [ ]:
! streamlit run my_app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.230.121.69:8501

your url is: https://every-peas-think.loca.lt
